# 8-семинар: Белгілер инжинирингі және логистикалық регрессия практикасы

**Семинар мақсаттары:**
1.  Классификация есебі үшін EDA жүргізу, сандық бағандарда жасырылған категориялық белгілерді анықтау.
2.  Категориялық белгілер үшін One-Hot Encoding қолдану.
3.  Сандық белгілерді масштабтау үшін `StandardScaler` қолдану.
4.  `GridSearchCV` арқылы гиперпараметрлерді автоматты түрде таңдайтын `LogisticRegression` моделін оқыту.
5.  Қателіктер матрицасын, классификация есебін және ROC-қисығын құрып, оларды егжей-тегжейлі түсіндіріп үйрену.

## 1. Деректерді жүктеу және зерттеу деректер талдауы (EDA)

Біз пациенттер туралы және оларда жүрек ауруының болуы туралы ақпаратты қамтитын `heart.csv` деректер жинағымен жұмыс істейтін боламыз.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/data/heart.csv')
df.head()

In [ ]:
df.info()

In [ ]:
sns.countplot(x='target', data=df)
plt.title('Мақсатты айнымалының таралуы')
plt.show()

**EDA-ның негізгі сәті:** `cp` (кеудедегі ауырсыну түрі), `thal` (тамырлардың жағдайы) және `slope` (ST сегментінің еңісі) сияқты кейбір белгілер сандармен берілген, бірақ шын мәнінде олар **категориялар** болып табылады. Сызықтық модельдер оларды реттелген сандық мәндер ретінде дұрыс түсіндірмейді. Сондықтан оларды түрлендіру қажет.

## 2. 1-қадам: Белгілер инжинирингі және деректерді дайындау

### 2.1. One-Hot Encoding

Біз категориялық белгілерге One-Hot Encoding қолданамыз. Бұл әрбір категория үшін жаңа бинарлы бағандар жасайды.

In [ ]:
X = df.drop('target', axis=1)
y = df['target']

# Категориялық болып табылатын бағандарды көрсетеміз
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

# pd.get_dummies қолданамыз
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

print("OHE-ден кейінгі X өлшемі:", X.shape)
X.head()

### 2.2. Таңдамаларға бөлу және масштабтау

Енді, барлық деректер сандық болған кезде, біз оларды оқыту және тест таңдамаларына бөліп, содан кейін масштабтауды қолдана аламыз.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
# scaler-ді ТЕК оқыту деректерінде оқытамыз
scaled_X_train = scaler.fit_transform(X_train)
# Оқытылған scaler-ді екі таңдамаға да қолданамыз
scaled_X_test = scaler.transform(X_test)

## 3. 2-қадам: Модельді оқыту

Біз `C` (реттеу күші) және `penalty` (реттеу түрі) гиперпараметрлерінің ең жақсы мәндерін автоматты түрде табу үшін `GridSearchCV` қолданамыз.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Модельді құрамыз
log_model = LogisticRegression(solver='liblinear')

# Гиперпараметрлер торын құрамыз
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-3, 3, 7)
}

# GridSearchCV нысанын құрамыз
grid_search = GridSearchCV(log_model, param_grid, cv=5, scoring='accuracy')

# Оқытамыз
grid_search.fit(scaled_X_train, y_train)

print("Үздік параметрлер:", grid_search.best_params_)

## 4. 3-қадам: Модельді бағалау

Енді GridSearchCV арқылы табылған ең жақсы модельді тест деректерінде бағалаймыз.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    RocCurveDisplay,
    classification_report
)

# Болжамдар
y_pred = grid_search.predict(scaled_X_test)

# Қателіктер матрицасы (plot_confusion_matrix орнына)
print("Қателіктер матрицасы:")
ConfusionMatrixDisplay.from_estimator(
    grid_search,            # fitted estimator (мысалы, GridSearchCV)
    scaled_X_test,
    y_test
)
plt.show()

# Классификация есебі
print("\nКлассификация есебі:")
print(classification_report(y_test, y_pred))

# ROC-қисығы (plot_roc_curve орнына)
# Егер модельде predict_proba немесе decision_function болса жұмыс істейді
print("\nROC-қисығы:")
RocCurveDisplay.from_estimator(grid_search, scaled_X_test, y_test)
plt.show()

### 3.4. Классификация есебін қалай оқу керек: практикалық нұсқаулық

Модель оқытылғаннан кейін, `scikit-learn` оны талдау үшін қуатты құралдар ұсынады: `confusion_matrix` және `classification_report`. Оларды қалай дұрыс түсіндіру керектігін қарастырайық.

#### **1. Қателіктер матрицасы (Confusion Matrix)**

Бұл сіздің моделіңіздің "рентгендік суреті". Ол модельдің қай жерде дұрыс, қай жерде қателесетінін көрсетеді.

|                | **Болжанған: 0** (Negative) | **Болжанған: 1** (Positive) |
|:---:|:---:|:---:|
| **Факт: 0** (Negative) | TN (True Negative) | FP (False Positive) |
| **Факт: 1** (Positive) | FN (False Negative) | TP (True Positive) |

- **TP (True Positive) — Ақиқат-оң нәтиже:** Модель **1** класын дұрыс болжады. *(Науқасты тапты)*.
- **TN (True Negative) — Ақиқат-теріс нәтиже:** Модель **0** класын дұрыс болжады. *(Сау адамды тапты)*.
- **FP (False Positive) — Жалған-оң нәтиже (I-түрдегі қате):** Модель **1** деп болжады, бірақ шын мәнінде **0** болды. *(Жалған дабыл: сау адамды науқас деп атады)*.
- **FN (False Negative) — Жалған-теріс нәтиже (II-түрдегі қате):** Модель **0** деп болжады, бірақ шын мәнінде **1** болды. *(Өткізіп алу: науқасты сау деп атады)*.

**Негізгі қорытынды:**
* Егер сіз үшін **жалған дабылдардан аулақ болу** өте маңызды болса (мысалы, спам-сүзгіде), сіз **FP**-ны азайтуға тырысуыңыз керек.
* Егер сіз үшін **ешнәрсені өткізіп алмау** өте маңызды болса (мысалы, медициналық диагностикада), сіз **FN**-ді азайтуға тырысуыңыз керек.

---

#### **2. Классификация есебі (classification_report)**

Бұл есеп қателіктер матрицасындағы ақпаратты талдауға ыңғайлы метрикаларға жинақтайды.

**Әр класс үшін метрикалар:**

* **Precision (Дәлдік)**
 <br>
 $$ Precision = \frac{TP}{TP + FP} $$
 <br>
 Модель оң деп атаған нысандардың қандай бөлігі **шынымен** сондай екенін көрсетеді. Жоғары дәлдік, егер модель "Иә" десе, оған сенуге болатынын білдіреді.

* **Recall (Толықтық)**
 <br>
 $$ Recall = \frac{TP}{TP + FN} $$
 <br>
 Модель **барлық нақты** оң нысандардың қандай бөлігін таба алғанын көрсетеді. Жоғары толықтық модельдің қажетті класты жақсы анықтайтынын білдіреді.

* **F1-score**
 <br>
 $$ F1 = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall} $$
 <br>
 `precision` мен `recall` арасындағы гармоникалық орташа. Бұл пайдалы жиынтық метрика, әсіресе теңгерімсіз класстар кезінде, себебі екі компоненттің бірі (precision немесе recall) өте төмен болса, ол да төмен болады.

* **Support**
 Тест таңдамасындағы әр кластың нақты нысандарының саны. Әрқашан осы санға қараңыз: егер қандай да бір класс үшін `support` өте аз болса (мысалы, 5-10 нысан), онда бұл класс үшін метрикалар статистикалық тұрғыдан сенімсіз болады.

**Жиынтық жолдар:**

* **Accuracy:** Барлық кластар бойынша дұрыс болжамдардың жалпы үлесі. Тек теңгерімді таңдамаларда ғана пайдалы.
* **Macro avg:** Барлық кластар бойынша метрикалардың орташа арифметикалық мәні. Әр класқа оның өлшеміне қарамастан бірдей салмақ береді. Модельдің "орташа есеппен барлық кластар бойынша" қаншалықты жақсы жұмыс істейтінін көрсетеді.
* **Weighted avg:** Әр кластың `support`-ын ескере отырып, метрикалардың орташа салмақталған мәні. Модельдің "орташа есеппен барлық нысандар бойынша" қаншалықты жақсы жұмыс істейтінін көрсетеді.

---

#### **3. ROC-қисығы және AUC**

ROC-қисығы шешім қабылдау шегі өзгерген кезде екі түрлі қателік арасындағы ымыраны көрсетеді.

* **TPR (True Positive Rate / Recall):** Дұрыс табылған оң нысандардың үлесі. $$ TPR = \frac{TP}{TP + FN} $$
* **FPR (False Positive Rate):** Жалған спрацовкалар үлесі. $$ FPR = \frac{FP}{FP + TN} $$

**AUC — ROC-қисығы астындағы аудан (Area Under the Curve):**
Бұл модельдің кластарды бөлу қабілетін жалпы бағалайтын бір сан.

* **0.9 - 1.0:** Керемет модель.
* **0.8 - 0.9:** Өте жақсы модель.
* **0.7 - 0.8:** Жақсы модель.
* **0.6 - 0.7:** Қанағаттанарлық модель.
* **0.5 - 0.6:** Нашар модель.
* **0.5:** Кездейсоқ болжау (пайдасыз модель).

---

#### **4. Табалдырықты таңдау (Threshold)**

Әдепкі бойынша, модель нысанды 1 класына жатқызады, егер оның ықтималдығы > `0.5` болса. Егер FP және FN қателерінің құны сіз үшін әртүрлі болса, бұл шекті өзгертуге болады.

* **Төмен шек (мысалы, 0.3):** **recall**-ды арттырады (көбірек оң нысандарды табамыз), бірақ **precision**-ды төмендетеді (көбірек жалған дабылдар).
* **Жоғары шек (мысалы, 0.7):** **precision**-ды арттырады (болжамдар өте сенімді), бірақ **recall**-ды төмендетеді (көбірек оң нысандарды өткізіп аламыз).

---

#### **5. Классификация моделін талдау чек-парағы**

1. `support`-қа қараңыз: Тест таңдамасындағы кластар теңгерімді ме?
2. **Қателіктер матрицасына** қараңыз: Модель қай жерде жиі қателеседі — **FP** немесе **FN**? Сіздің бизнес-есебіңіз үшін қай қате "ауырырақ"?
3. Қызықтыратын класс үшін **`precision`** мен **`recall`**-ды салыстырыңыз: Олардың арасында қатты айырмашылық бар ма?
4. **`macro avg`** мен **`weighted avg`**-ды салыстырыңыз: Егер олар қатты ерекшеленсе, демек, модель сирек кездесетін класпен нашар жұмыс істейді.
5. **`AUC`**-ке қараңыз: Модель негізінен кластарды қаншалықты жақсы бөле алады?
6. Қажет болса (егер қателердің құны әртүрлі болса) — сіздің есебіңізге `precision` мен `recall`-ды теңгеру үшін оңтайлы **шекті** таңдаңыз.

* **AUC ≈ 0.93**

---

#### **Чек-парақ бойынша талдау:**

1. **`support` (кластар теңгерімі):**
 * Класс 0: 29 нысан.
 * Класс 1: 32 нысан.
 * **Қорытынды:** Кластар іс жүзінде өте жақсы теңгерілген. Бұл `accuracy` метрикасының сенімді екенін және бізді адастырмайтынын білдіреді.

2. **Кластар бойынша `precision` және `recall`:**
 * **Класс 0:** `precision` = 0.86, `recall` = 0.86. Модель бұл класс үшін жалған спрацовкалардан аулақ болуды да, өткізіп алуды да бірдей жақсы орындайды.
 * **Класс 1:** `precision` = 0.88, `recall` = 0.88. Модель екінші класс үшін де теңгерілген дәлдік пен толықтықты көрсетеді.
 * **Қорытынды:** `precision` мен `recall` арасында ешбір класс үшін қатты айырмашылық жоқ, бұл модельдің тұрақты жұмысын көрсетеді.

3. **`macro avg` және `weighted avg` салыстыру:**
 * `macro avg f1-score` = 0.87.
 * `weighted avg f1-score` = 0.87.
 * **Қорытынды:** Мәндер іс жүзінде бірдей, бұл кластардың жақсы теңгерілгендігі туралы қорытындыны тағы да растайды. Модель бір класс үшін сапаны екіншісі үшін "құрбан етпейді".

4. **`AUC` (бөлу сапасы):**
 * **AUC ≈ 0.93**.
 * **Қорытынды:** Бұл өте жоғары мән. Бұл модельдің екі кластың нысандарын **өте жақсы бөлетінін** білдіреді. Жоғары ықтималдылықпен ол кездейсоқ таңдалған 1-класс нысанына кездейсоқ таңдалған 0-класс нысанына қарағанда жоғары баға (ықтималдық) береді.

---

#### **Модель бойынша қысқаша түйіндеме**

* **Тұрақтылық:** Модель екі класста да айқын ауытқуларсыз бірдей жақсы жұмыс істейді.
* **Бөлу қабілеті:** Модель бір класты екіншісінен сенімді түрде ажыратады (жоғары `AUC`).
* **Жалпы сапа:** Жоғары (Accuracy ≈ 87%, F1 ≈ 87%).

**Келесі қадам:** Әдепкі бойынша, модель `0.5` шегімен жұмыс істейді, бұл `precision` мен `recall` арасында жақсы теңгерім береді. Егер бизнес-есеп үшін **False Negative** (науқасты өткізіп алу) қатесінің құны **False Positive** (жалған дабыл) қатесінің құнынан әлдеқайда жоғары болса, онда келесі қадам `recall`-ды арттыру үшін **төменірек шекті таңдау** (мысалы, `0.4`) болуы мүмкін.